In [2]:
import tensorflow as tf 

# Download and import the MIT 6.S191 package
# !pip install mitdeeplearning
# import mitdeeplearning as mdl

# Import all remaining packages
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm

from IPython.display import Audio
# from pretty_midi import PrettyMIDI

In [3]:
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [4]:
# for root, dirs, files in os.walk("./data", topdown=False):
#    for name in files:
#       print(os.path.join(root, name))
#    for name in dirs:
#       print(os.path.join(root, name))


# import pygame

# def play_music(music_file):
#     """
#     stream music with mixer.music module in blocking manner
#     this will stream the sound from disk while playing
#     """
#     clock = pygame.time.Clock()
#     try:
#         pygame.mixer.music.load(music_file)
#         print( "Music file %s loaded!" % music_file)
#     except pygame.error:
#         print ("File %s not found! (%s)" % (music_file, pygame.get_error()))
#         return
#     pygame.mixer.music.play()
#     while pygame.mixer.music.get_busy():
#         # check if playback has finished
#         clock.tick(30)
# # pick a midi music file you have ...
# # (if not in working folder use full path)

# midi_file = data['Chamber2_MID--AUDIO_09_R3_2018_wav--1']
# freq = 44100    # audio CD quality
# bitsize = -16   # unsigned 16 bit
# channels = 2    # 1 is mono, 2 is stereo
# buffer = 1024    # number of samples
# pygame.mixer.init(freq, bitsize, channels, buffer)

# # optional volume 0 to 1.0
# pygame.mixer.music.set_volume(0.8)
# try:
#     play_music(midi_file)
# except KeyboardInterrupt:
#     # if user hits Ctrl/C then exit
#     # (works only in console mode)
#     pygame.mixer.music.fadeout(1000)
#     pygame.mixer.music.stop()
#     raise SystemExit

In [5]:
data = {}
file = ""
midi = False
for i, (dirpath, dirnames, filenames) in enumerate(os.walk('./data', topdown=False)):
#     print(i, dirpath, dirnames, filenames)
    for f in filenames: #f = name of file, not full file path 
        
        file_path = os.path.join(dirpath, f)
#         print(file_path)
#         print(f)
#         print(f[17:-5])
            
        if "midi" in f:
            file = f[17:-5]
#             print(f)
        data[file] = file_path

# for key, val in data.items():
#     print(key)
#     print("--> ", val)
#     print()

# print(data)

In [6]:
data['Chamber2_MID--AUDIO_09_R3_2018_wav--1']

'./data\\2018\\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1.midi'

In [7]:
notes = []
keys = data.keys()

for key in data.keys():
    file = data[key]
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [8]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

In [9]:
len(notes)
notes

['A5',
 'F#4',
 'D4',
 'A4',
 'F#4',
 'A4',
 'F#4',
 'D6',
 'A5',
 'A4',
 'D4',
 'F#5',
 'F#4',
 'A4',
 'F#4',
 'A4',
 'A5',
 'F#4',
 'A4',
 'F#5',
 'D5',
 'A3',
 'F#4',
 'A4',
 'D5',
 'F#4',
 'A4',
 'E5',
 '1.7',
 'F#5',
 'E5',
 'F#5',
 'A4',
 'E5',
 'D5',
 'E5',
 'F#5',
 'D4',
 'F#4',
 'D5',
 'A4',
 'A5',
 'F#4',
 'D4',
 'A4',
 'F#4',
 'A4',
 '2.6',
 'A4',
 'A5',
 'F#4',
 'F#5',
 'D4',
 'A4',
 'F#4',
 'A4',
 'A5',
 'F#4',
 'A4',
 'F#5',
 'A3',
 'D5',
 'F#4',
 'A4',
 'D5',
 'F#4',
 'A4',
 'F#5',
 'E5',
 '1.7',
 'F#5',
 'E5',
 'A4',
 'F#5',
 'D5',
 '2.6',
 'A3',
 'D3',
 'D4',
 'F#4',
 'A5',
 'A4',
 'F#4',
 'A4',
 'D6',
 'F#4',
 'A5',
 'A4',
 'F#5',
 'D4',
 'F#4',
 'A4',
 'F#4',
 'A4',
 'A5',
 'F#4',
 'A4',
 'F#5',
 'D5',
 'A3',
 'F#4',
 'A4',
 'D5',
 'F#4',
 'A4',
 'E5',
 '1.7',
 'F#5',
 'E5',
 'F#5',
 'A4',
 'E5',
 'D5',
 'E5',
 'F#5',
 '2.6',
 'D5',
 'A4',
 'A5',
 'F#4',
 'D4',
 'A4',
 'F#4',
 'A4',
 '2.6',
 'A4',
 'A5',
 'D4',
 'F#5',
 'F#4',
 'A4',
 'F#4',
 'A4',
 'A5',
 'F#4',
 'A

In [13]:
sequence_length = 100
# get all pitch names
n_vocab = len(set(notes))
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [14]:
print(set(notes))

{'4.7.9', 'G1', 'G6', '5.9.11', 'D4', '7.10.2', '0.4.7', '0.2', '6.10.0', '11.2.4', '8.10.0', 'C2', '4.6.10', 'G4', '7.0', '4.7.10', '9.0.4', 'F#4', '2.6.7', 'D3', 'B-1', '4.8', '1', '0.3', 'B5', 'B1', '7.10.1', '0.1', '0.6', '0.1.4', 'C5', 'C#6', '5', '3.4', '9.0', 'B-4', '6.9', '9', '3.9', '8.0.2', '2.3.8', 'A1', 'F#3', '6.9.1', '3.5', '2.4.7', '8.9', 'B-3', '9.2', '4.6', '11.2.5', '8', '2.5', '10.2', 'A6', 'E-3', '1.3', '0.5', '5.7', '11.0', '7.11.2', 'F#6', 'A5', '3.8', '2.4.6', '7.9.10.2', 'C#5', '10.3', 'F4', '0', '9.10.1', '2.4', 'F3', '0.3.7', '9.1', '11.1', 'F5', 'C4', '1.4.7', '9.1.4', 'E3', '3.6.10', '2.3.7', '4.6.9.11', '9.11', '8.11', 'G#4', '2.3', '4.7.11', 'F#7', 'B2', 'G5', '5.10', 'E5', 'E6', '6.7', '8.0', '6.7.11', 'E4', '6.11', '3.5.6.10', '1.2', '7.9', 'C#4', '3.5.8.11', '4.7', 'B6', 'G#1', '8.1', 'B4', 'F6', '6.8.1', '3.7.11', '3.6.8', 'D2', 'G#2', '10.11', '1.4.7.9', 'E-5', '5.6', 'A3', 'C6', '2.7', '0.4', '7.10', '11.4', '11.2.4.7', '10.0', '0.2.6', '10.0.4', '7.

In [15]:
network_output.shape

(7720, 214)

In [16]:
model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [18]:
filepath = "weights.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=2, batch_size=64, callbacks=callbacks_list)

Epoch 1/2
  2/121 [..............................] - ETA: 5:05 - loss: 5.3135 

KeyboardInterrupt: 

In [17]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights.hdf5')

ValueError: Cannot assign to variable lstm_9/lstm_cell_9/kernel:0 due to variable shape (1, 2048) and value shape (1, 1024) are incompatible

In [ ]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')